# Similar objects in simbad, GEV and TEV catalogs
First, we import all the packages that we will need.

In [1]:
import warnings
import pandas as pd
import numpy as np
from getsimbad import get_simbad_data
from gevtev import compare_gev_tev_data

warnings.filterwarnings("ignore")

This constant determines a threshold for the similarity between the objects:
$$|glat_{1} - glat_{2}| <= \epsilon$$
$$\epsilon = 1,2*10^{-1}$$

In [2]:
_epsilon = 1.2e-1

The cell above loads data from SIMBAD catalog obtained by requests 'Be', 'O', 'B' and 'plsr'. If the file "data/symbad.txt" does not exist, run getsimbad.py.

In [3]:
try:
    pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', encoding='utf-8')
    del pd_simbad['Unnamed: 0']
except FileNotFoundError:
    pd_simbad = get_simbad_data()


In [4]:
pd_simbad.head()

,s_MAIN_ID,s_RA,s_DEC,s_RA_PREC,s_DEC_PREC,s_COO_ERR_MAJA,s_COO_ERR_MINA,s_COO_ERR_ANGLE,s_COO_QUAL,s_COO_WAVELENGTH,...,s_FLUX_H,s_FLUX_K,s_FLUX_u,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,s_class,s_glat,s_glon
0,b'HD 201433A',21 08 38.894,+30 12 20.27,7.0,7.0,26.0000,26.0000,90.0,B,O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,plsr,-11.774311,76.107743
1,b'* nu. For',02 04 29.4462,-29 17 48.491,14.0,14.0,0.1834,0.1942,90.0,A,O,...,5.048,5.015,NaN,NaN,NaN,NaN,NaN,plsr,-73.777993,224.887601
2,b'* phi Her',16 08 46.1668,+44 56 05.732,14.0,14.0,0.3710,0.3754,90.0,A,O,...,4.330,4.320,NaN,NaN,NaN,NaN,NaN,plsr,47.107400,70.847825
3,b'V* CG And',00 00 43.6345,+45 15 12.002,14.0,14.0,0.0550,0.0422,90.0,A,O,...,6.506,6.511,NaN,NaN,NaN,NaN,NaN,plsr,-16.702455,113.649358
4,b'HD 133029',15 00 38.7184,+47 16 38.793,14.0,14.0,0.0472,0.0421,90.0,A,O,...,6.585,6.570,NaN,NaN,NaN,NaN,NaN,plsr,57.710074,80.191942


We load data common for GEV and TEV catalogs.

In [5]:
common_data, only_tev_data, only_gev_data = compare_gev_tev_data(_epsilon)

The following cells compare distances from a common catalog of GEV and TEV and SIMBAD catalog by translating SIMBAD coordinates to glan and glot due to astropy module. If their relative distance is smaller than $\epsilon$ they are associated to each another.

In [6]:
def create_matrix_common_gevtev_simbad(gevtev, simbad, epsilon, catalog="tev"):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a matrix of associations GeVTeV - Simbad
    (m[i][j] = True if an object i from GeVTeV has coordinates that are close to ones of an object j from simbad)
    
    gevtev - a pandas dataFrame of common GeV and TeV 
    simbad - a pandas dataFrame of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    if (catalog.find("tev") != -1):
        glat_gevtev = gevtev[catalog+'_glat']
        glon_gevtev = gevtev[catalog+'_glon']
    else:
        glat_gevtev = gevtev[catalog+'_GLAT']
        glon_gevtev = gevtev[catalog+'_GLON']        

    glat_simbad = simbad['s_glat']
    glon_simbad = simbad['s_glon']
    
    C_associations_gevtev = -1.0 * np.ones((len(glat_gevtev)))
    C_associations_simbad = -1.0 * np.ones((len(glat_simbad)))
    
    glat_dif_matrix = np.dot(np.vstack((glat_gevtev, -np.ones_like(glat_gevtev))).T,
                            np.vstack((np.ones_like(glat_simbad), glat_simbad)))
    glon_dif_matrix = np.dot(np.vstack((glon_gevtev, -np.ones_like(glon_gevtev))).T,
                            np.vstack((np.ones_like(glon_simbad), glon_simbad)))
    pairs_matrix = np.logical_and(np.abs(glat_dif_matrix) < epsilon, 
                                np.abs(glon_dif_matrix) < epsilon)
    
    for i in range(C_associations_gevtev.size):
        associations = np.argwhere(pairs_matrix[i])
        if associations.size > 0:
            C_associations_gevtev[i] = associations.max()
            
    for j in range(C_associations_simbad.size):
        associations = np.argwhere(pairs_matrix[:, j])
        if associations.size > 0:
            C_associations_simbad[j] = associations.max()

    return pairs_matrix

In [7]:
def create_common_data(data_gevtev, data_simbad, pairs_matrix, catalog="tev"):
    """The fonction adds objects found both in GeV, TeV and simbad.
    """
    vector_association = np.where(np.sum(pairs_matrix, axis=0) > 0)[0]
    pd_common_gevtevsimbad = pd.DataFrame()
    for i in vector_association:
        data_gevtev_join = (pairs_matrix[:, i] > 0)*(i + 1) - 1
        data_gevtev["join"] = data_gevtev_join
        pd_common_gevtevsimbad0 = pd.merge(data_gevtev, data_simbad, right_index=True, left_on='join', how='inner')
        if (len(pd_common_gevtevsimbad)):
            pd_common_gevtevsimbad = pd_common_gevtevsimbad.append(pd_common_gevtevsimbad0)
            del pd_common_gevtevsimbad["join"]
        else:
            pd_common_gevtevsimbad = pd_common_gevtevsimbad0.copy()

    #df_common = pd.DataFrame(data = data, columns = namefinal)   
    if (catalog.find("tev") != -1):    
        array_non_duplicate = [catalog+'_glat', catalog+'_glon', 's_glat','s_glon']
    else:
        array_non_duplicate = [catalog+'_GLAT', catalog+'_GLON', 's_glat','s_glon']
    pd_common_gevtevsimbad = pd_common_gevtevsimbad.drop_duplicates(array_non_duplicate)
    pd_common_gevtevsimbad = pd_common_gevtevsimbad.reset_index()
    
    #df_common = pd.DataFrame(data = data, columns = namefinal)
    return pd_common_gevtevsimbad

In [8]:
pairs_matrix = create_matrix_common_gevtev_simbad(common_data, pd_simbad, _epsilon)
pd_common_gevtevsimbad = create_common_data(common_data, pd_simbad, pairs_matrix)
print("There are " + str(len(pd_common_gevtevsimbad)) + " common objects in the three catalogs.")

There are 68 common objects in the three catalogs.


In [9]:
pd_common_gevtevsimbad.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_pl_norm_err,tev_spec_pl_norm_err_sys,tev_spec_theta,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,1.300000e-12,2.000000e-13,0.70,b'pl',b'amNIwH',142,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal'
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,5.000000e-14,NaN,0.09,b'pl',b'LsZtAg',277,b'TeV J1746-289',-9223372036854775808,b'',b'gal'
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'


The next cell chooses several columns from a dataFrame to make sure that the output is correct(objects have the same coordinates etc.)

In [10]:
columns_to_compare = ['s_glat',
                     's_glon',
                     'gev_ASSOC_TEV',
                     'tev_glat', 
                     'gev_GLAT',
                     'tev_glon',
                     'gev_GLON',
                     's_class',
                     'gev_CLASS1',
                     'tev_classes']
pd_common_gevtevsimbad[columns_to_compare].head()

,s_glat,s_glon,gev_ASSOC_TEV,tev_glat,gev_GLAT,tev_glon,gev_GLON,s_class,gev_CLASS1,tev_classes
0,-0.011453,359.985525,b' ',-0.046162,-0.020071,359.944244,359.863953,Be,b' ',b'unid'
1,-0.198384,26.857782,b'HESS J1841-055 ',-0.197604,-0.198590,26.795113,26.795933,B,b'PWN ',b'unid'
2,-0.047327,359.945163,b' ',-0.046162,-0.020071,359.944244,359.863953,B,b' ',b'unid'
3,-0.070598,0.237663,b' ',-0.113206,-0.103157,0.140617,0.149201,O,b'pwn ',b'unid'
4,-0.042219,359.951421,b' ',-0.046162,-0.020071,359.944244,359.863953,O,b' ',b'unid'


In [11]:
columns_to_compare_gevtev = ['gev_ASSOC_TEV',
                     'tev_glat', 
                     'gev_GLAT',
                     'tev_glon',
                     'gev_GLON',
                     'gev_CLASS1',
                     'tev_classes']

The next cells find common values in simbad-tev

In [12]:
pairs_matrix = create_matrix_common_gevtev_simbad(only_tev_data, pd_simbad, _epsilon)
pd_common_onlytevsimbad = create_common_data(only_tev_data, pd_simbad, pairs_matrix, catalog="tev")
print("There are " + str(len(pd_common_onlytevsimbad)) + " common objects in tev-simbad catalogs.")

There are 123 common objects in tev-simbad catalogs.


The next cells find common values in simbad-gev

This cell does not take up to 40 minutes anymore :)

In [13]:
pairs_matrix = create_matrix_common_gevtev_simbad(only_gev_data, pd_simbad, _epsilon, catalog="gev")
pd_common_onlygevsimbad = create_common_data(only_gev_data, pd_simbad, pairs_matrix, catalog="gev")
print("There are " + str(len(pd_common_onlygevsimbad)) + " common objects in gev-simbad catalogs.")

There are 155 common objects in gev-simbad catalogs.


In [14]:
pd_common_gevtevsimbad.to_csv("data/gevtevsimbad.txt", sep='\t')
pd_common_onlygevsimbad.to_csv("data/gevsimbad.txt", sep='\t')
pd_common_onlytevsimbad.to_csv("data/tevsimbad.txt", sep='\t')
common_data.to_csv("data/gevtev.txt", sep='\t')

In [15]:
only_gev_data.to_csv("data/gev.txt", sep='\t')
only_tev_data.to_csv("data/tev.txt", sep='\t')
pd_simbad.to_csv("data/simbad.txt", sep='\t')